In [1]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn import linear_model
import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn import ensemble
import time
import random
import gensim
from gensim.models import doc2vec
from collections import namedtuple

/Users/s0c01xa/Desktop/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
mydata=pd.read_csv('/Users/s0c01xa/Documents/Kaggle/Lord of machines.csv')
mydata=np.array(mydata)

In [5]:
data=mydata[:,1]
docLabels=mydata[:,0]
type(data)

numpy.ndarray

# DOC TO VEC USING GENSIM

In [6]:
# Load data

#data saves the documents and doclables has just the labels for the corresponding docs
#docLabels has just the ids of the documents
data=mydata[:,1]
docLabels=mydata[:,0]
# Transform data (you can add more data preprocessing steps) 

#Creating a tuple since gensim takes it in the form of tuples,where there will be the doc
#attached will be the label
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')

#converting it to int type
'''for i in range(0,len(docLabels)):
    docLabels[i]=int(docLabels[i])'''

#converting the words to lowercase and appending it in docs in the form of tuples
#docs[0] will contain the tuple of first doc and it's label
for i in range (0,len(data)):
    words = data[i].lower().split()
    tags=[str(docLabels[i])]
    docs.append(analyzedDocument(words, tags))



# DOC TO VEC CONVERSION USING GENSIM

In [613]:
from gensim.models import doc2vec
from collections import namedtuple

# Train model (set min_count = 1, if you want the model to work with the provided example data set)
model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

#model.docvecs[i] saves the corresponding vector for the document with label numbers mentioned in tags


In [614]:
len(model.docvecs)

52

In [615]:
train_clust_data=pd.read_csv('/Users/s0c01xa/Documents/train_HFxi8kT/train.csv')
campaign_data=pd.read_csv('/Users/s0c01xa/Documents/train_HFxi8kT/campaign_data.csv')
#train_clust_data=np.array(train_clust_data)
#train_clust_data

In [616]:
comb_data= train_clust_data[['user_id','campaign_id','is_open']]
#comb_data2=train_clust_data[['user_id','campaign_id','is_click']]
#camp_id= train_clust_data[:,1]
#is_open=train_clust_data[:,2]


In [618]:
comb_data2=train_clust_data[['user_id','campaign_id','is_click']]

In [619]:
#Pandascrosstab output checking
df = train_clust_data.pivot(index = 'user_id', columns ='campaign_id', values = 'is_open').fillna(0)
userId = df.index
#df=np.array(df)

In [620]:
df=np.array(df)
userId=np.array(userId)

# KMEANS

In [621]:
#Clustering algorithm on df to cluster the set of users
#No of clusters n_clust will vary and will be adjusted 
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=1).fit(df)

In [622]:
#Labels it will store the entire labels for each of the 
clust_labels=kmeans.labels_

In [623]:
#f_data stores the user id in the first column and the corr. cluster labels in the second column
f_data=np.column_stack((userId,clust_labels))
#f_data=pd.DataFrame(f_data)

In [203]:
#It saves the length of the ids with the corresponding clust labels.
#len(f_data[clust_labels==i][:,0])

2196

In [624]:
f_data=pd.DataFrame(f_data)
#f_data

# VLOOKUP IN PYTHON

In [625]:
#FIT CLASSIFICATION MODELS FOR EACH CLUSTER
#which is identified clust_labels==i
#Implementing a vlookup in python
#first element is the column no where this new column will be added
#second element is the column name in the first sheet which needs to be filled
#third element is the mapping in the second sheet and the name of the column to be mapped

#adding columns to python
#comb_data.insert(3, 'communication_type', comb_data['campaign_id'].map(campaign_data.set_index('campaign_id')['communication_type']))
#comb_data.insert(4, 'subject', comb_data['campaign_id'].map(campaign_data.set_index('campaign_id')['subject']))



In [627]:
#comb_data stores the entire information with the users and the corresponding subject and comm_type
#which is needed to predict is_open


# DOC TO VEC FOR CAMPAIGN ID OF TRAINING DATA  BASED ON SUBJECT


In [628]:
# Load data

#data saves the documents and doclables has just the labels for the corresponding docs
#docLabels has just the ids of the documents
data=mydata[:,1]
docLabels=mydata[:,0]
# Transform data (you can add more data preprocessing steps) 

#Creating a tuple since gensim takes it in the form of tuples,where there will be the doc
#attached will be the label
docs = []
w2v=np.zeros(52)
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')

#converting it to int type
'''for i in range(0,len(docLabels)):
    docLabels[i]=int(docLabels[i])'''

#converting the words to lowercase and appending it in docs in the form of tuples
#docs[0] will contain the tuple of first doc and it's label
for i in range (0,len(data)):
    words = data[i].lower().split()
    tags=[str(docLabels[i])]
    docs.append(analyzedDocument(words, tags))
    w2v[i]=str(docLabels[i])

In [629]:
# Train model (set min_count = 1, if you want the model to work with the provided example data set)
model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)
#model.docvecs[i] saves the corresponding vector for the document with label numbers mentioned in tags



In [630]:
word2vec=np.zeros((52,100))
for i in range(0,len(w2v)):
    word2vec[i]=model.docvecs[i]


In [631]:
#final word to vec is being saved with corresponding campaign id and is stored in word2vec_f
word2vec_f=np.column_stack((w2v,word2vec))
word2vec_f=pd.DataFrame(word2vec_f)

# VLOOKUP-2


In [632]:
#first element is the column no where this new column will be added
#second element is the column name in the first sheet which needs to be filled
#third element is the mapping in the second sheet and the name of the column to be mapped

#for loop loops through and adds the number of columns we need, here the vectors are added in comb_data
for i in range(0,100):
    comb_data.insert(i+5, i, comb_data['campaign_id'].map(word2vec_f.set_index(0)[i+1]))
    
#Vectors are mapped and saved in comb_data-main document

In [634]:
x1=comb_data.drop(comb_data[['campaign_id', 'is_open', 'subject']], axis=1)
y1=comb_data['is_open']

In [635]:
#Encoding the communication type
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
communication_type=le.fit_transform(x1['communication_type'])
x1.insert(102,'comm_type',communication_type)


# VLOOKUP-3


In [679]:
#Now the cluster labels are also present inside the data x1. We need to filter by Clust_labels and do the classif
#save the model
#x1.drop(x1[['communication_type']], axis=1)
x1=x1.drop(x1[['Clust_labels','is_open']], axis=1)
#x1.insert(102, 'Clust_labels', comb_data['user_id'].map(f_data.set_index(0)[1]))
#x1.insert(103,'is_open',y1)

#x1 and y1 now contains the important features for the first phase i.e for is_open

In [681]:
x1.insert(103, 'Clust_labels', comb_data['user_id'].map(f_data.set_index(0)[1]))
x1.insert(104,'is_open',y1)

# DOC TO VEC FOR CAMPAIGN ID OF TRAINING DATA  BASED ON BODY

In [640]:
# Load data

#data saves the documents and doclables has just the labels for the corresponding docs
#docLabels has just the ids of the documents

campaign_data=pd.read_csv('/Users/s0c01xa/Documents/train_HFxi8kT/campaign_data.csv')

data2=campaign_data['email_body']
docLabels2=campaign_data['campaign_id']

# Transform data (you can add more data preprocessing steps) 

#Creating a tuple since gensim takes it in the form of tuples,where there will be the doc
#attached will be the label
docs2 = []
w2v2=np.zeros(52)
analyzedDocument2 = namedtuple('AnalyzedDocument', 'words tags')

#converting the words to lowercase and appending it in docs in the form of tuples
#docs[0] will contain the tuple of first doc and it's label
for i in range (0,len(data2)):
    words = data2[i].lower().split()
    tags=[str(docLabels2[i])]
    docs2.append(analyzedDocument(words, tags))
    w2v2[i]=str(docLabels2[i])

In [641]:
# Train model (set min_count = 1, if you want the model to work with the provided example data set)
model2 = doc2vec.Doc2Vec(docs2, size = 100, window = 300, min_count = 1, workers = 4)
#model2.docvecs[i] saves the corresponding vector for the document with label numbers mentioned in tags



In [644]:
body2vec=np.zeros((52,100))
for i in range(0,len(w2v2)):
    body2vec[i]=model2.docvecs[i]

#final word to vec is being saved with corresponding campaign id and is stored in word2vec_f
body2vec_f=np.column_stack((w2v2,body2vec))
body2vec_f=pd.DataFrame(body2vec_f)

In [646]:
#body2vec_f

In [647]:
#Adding important features for predicting is_click

comb_data2.insert(3, 'total_links', comb_data2['campaign_id'].map(campaign_data.set_index('campaign_id')['total_links']))
comb_data2.insert(4, 'no_of_internal_links', comb_data2['campaign_id'].map(campaign_data.set_index('campaign_id')['no_of_internal_links']))
comb_data2.insert(5, 'no_of_images', comb_data2['campaign_id'].map(campaign_data.set_index('campaign_id')['no_of_images']))
comb_data2.insert(6, 'no_of_sections', comb_data2['campaign_id'].map(campaign_data.set_index('campaign_id')['no_of_sections']))

In [544]:
#Adding body2vec elements in combdata2

In [651]:
#comb_data2

In [652]:
#first element is the column no where this new column will be added
#second element is the column name in the first sheet which needs to be filled
#third element is the mapping in the second sheet and the name of the column to be mapped

#for loop loops through and adds the number of columns we need, here the vectors are added in comb_data

for i in range(0,100):
    comb_data2.insert(i+7, i, comb_data2['campaign_id'].map(body2vec_f.set_index(0)[i+1]))

In [654]:
#comb_data2

In [655]:
x2=comb_data2.drop(comb_data2[['campaign_id', 'is_click']], axis=1)
y2=comb_data2['is_click']

In [661]:
#x2
#comb_data2

In [662]:
x2.insert(105, 'Clust_labels', comb_data2['user_id'].map(f_data.set_index(0)[1]))
#x2.insert(106, 'Clust_labels', comb_data2['user_id'].map(f_data.set_index(0)[1]))

In [664]:
#x2 contains the features needed to model for the isclick response
#y2 response of isclick
#x2

In [571]:
#x2

# MODEL BUILDING TWO LEVEL

In [733]:
import pickle
from sklearn.linear_model import LogisticRegression as lg

model1=lg(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None)
model2=lg(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None)


file_model1="model"


for i in range(0,20):
    y = x1.loc[x1['Clust_labels']==i]
    feat=y.iloc[:,2:102]
    res=y.iloc[:,-1]
    model1.fit(feat,res)
    s = pickle.dump(model, open(file_model1+str(i), 'w'))
    #isopen_model1.append("isopen_model1"+str(i))
    #feat1 will have the probabilities of opening a mail
    feat1_isclick=model1.predict_proba(feat)[:,1]
    #Features for model2 needs to be predicted
    y2 = x2.loc[x2['Clust_labels']==i]
    feat1=y2.iloc[:,1:105]
    res1=y2.iloc[:,-1]
    feat1.insert(104, 'is_open_prob',feat1_isclick)
    model2.fit(feat1,res1)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

# USE FUZZY AUGMENTATION/ANY OTHER AUGMENTATION TECHNIQUE

In [746]:
#train_clust_data['is_click']

In [737]:
y2 = x2.loc[x2['Clust_labels']==0]
feat1=y2.iloc[:,1:105]
#res1=y2.iloc[:,-1]
#feat1.insert(104, 'is_open_prob',feat1_isclick)
#y2

#len(feat1)

In [738]:
model1=lg(penalty='l2', dual=False, tol=0.001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None)
y=x1.loc[x1['Clust_labels']==1]
feat=y.iloc[:,2:102]
res=y.iloc[:,-1]

model1.fit(feat,res)

feat1_isclick=model1.predict_proba(feat)[:,0]

In [692]:
#cross_val_score(model1,feat,res,cv=10)

In [752]:
import pickle
from sklearn.linear_model import LogisticRegression as lg

model1=lg(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None)
model2=lg(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None)

 
#file_model="/Users/s0c01xa/Documents/models/model"


for i in range(0,20):
    y = x1.loc[x1['Clust_labels']==i]
    feat=y.iloc[:,2:102]
    res=y.iloc[:,-1]
    model.fit(feat,res)
    #the file name and location is specified
    #In the folder pklmodel,with the model number corr to cluster numbers,the models are being stored.
    #Here the strings/names are made
    filename = "/Users/s0c01xa/Documents/pkl_models/model"+'_'+str(i+1)+'.pkl'
    #Open the filename to write 'w'
    file_object = open(filename, 'w')
    #dumps the model
    s = pickle.dump(model, file_object)
    #close the file
    file_object.close()
    
#pkl model stores all the model, now for test data we can use the same.


In [753]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [756]:
y = x1.loc[x1['Clust_labels']==10]
feat=y.iloc[:,2:102]
res=y.iloc[:,-1]
cross_val_score(model,feat,res,cv=10)

array([ 0.81809045,  0.81790744,  0.81991952,  0.81690141,  0.81589537,
        0.81772407,  0.80664653,  0.81873112,  0.82678751,  0.82477341])